In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
import mlflow
# mlflow ui --backend-store-uri sqlite:///mlflow.db
# mlflow server --host 127.0.0.1 --port 5000
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("drinking")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

SyntaxError: invalid syntax (4194309078.py, line 2)

In [4]:
mlflow.autolog()

In [5]:
tqdm.pandas()

In [6]:
path = 'Artefacts/'

In [7]:
df = pd.read_parquet("Data/smoking_drinking.parquet")

In [ ]:
df

In [ ]:
df.info()

In [8]:
df.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,...,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000
mean,47.614491,162.240625,63.284050,81.233358,0.980834,0.978429,1.031495,1.030476,122.432498,76.052627,...,56.936800,113.037692,132.141751,14.229824,1.094224,0.860467,25.989308,25.755051,37.136347,1.608122
std,14.181339,9.282957,12.514241,11.850323,0.605949,0.604774,0.174650,0.171892,14.543148,9.889365,...,17.238479,35.842812,102.196985,1.584929,0.437724,0.480530,23.493386,26.308599,50.424153,0.818507
min,20.000000,130.000000,25.000000,8.000000,0.100000,0.100000,1.000000,1.000000,67.000000,32.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000
25%,35.000000,155.000000,55.000000,74.100000,0.700000,0.700000,1.000000,1.000000,112.000000,70.000000,...,46.000000,89.000000,73.000000,13.200000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.000000,120.000000,76.000000,...,55.000000,111.000000,106.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000
75%,60.000000,170.000000,70.000000,87.800000,1.200000,1.200000,1.000000,1.000000,131.000000,82.000000,...,66.000000,135.000000,159.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000
max,85.000000,190.000000,140.000000,999.000000,9.900000,9.900000,2.000000,2.000000,273.000000,185.000000,...,8110.000000,5119.000000,9490.000000,25.000000,6.000000,98.000000,9999.000000,7210.000000,999.000000,3.000000


In [9]:
df[df.isnull().any(axis=1)]

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN


In [10]:
df[df.duplicated].shape

(26, 24)

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df[df.duplicated].shape

(0, 24)

In [13]:
def remove_outliers(df, columns, k=1.5):
    for column in columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75) 
        iqr = q3 - q1
        df[column] = df[column].clip(lower=q1 - k * iqr, upper=q3 + k * iqr)

    return df

In [14]:
remove_outliers(df,['waistline','SBP', 'DBP','BLDS','tot_chole','triglyceride','serum_creatinine','SGOT_AST', 'SGOT_ALT'],k=1.5)

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,41.5,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991341,Male,45,175,80,92.1,1.5,1.5,1.0,1.0,114.0,...,125.0,132.0,15.0,1.0,1.0,26.0,36.0,27.0,1.0,N
991342,Male,35,170,75,86.0,1.0,1.5,1.0,1.0,119.0,...,84.0,45.0,15.8,1.0,1.1,14.0,17.0,15.0,1.0,N
991343,Female,40,155,50,68.0,1.0,0.7,1.0,1.0,110.0,...,77.0,157.0,14.3,1.0,0.8,30.0,27.0,17.0,3.0,Y
991344,Male,25,175,60,72.0,1.5,1.0,1.0,1.0,119.0,...,73.0,53.0,14.5,1.0,0.8,21.0,14.0,17.0,1.0,N


In [15]:
df['sex'] = np.where(df['sex'].values=='Female',0,1)

In [16]:
df['DRK_YN'] = np.where(df['DRK_YN'].values=='N',0,1)

In [17]:
def get_bmi(row):
    weight = row["weight"]
    height = row["height"]
    return weight / (height ** 2)

In [18]:
def get_blood_pressure_ratio(row):
    sbp = row["SBP"]
    dbp = row["DBP"]
    return sbp / dbp

In [19]:
def get_hdl_cholestorol_ratio(row):
    hdl_chole = row["HDL_chole"]
    total_chole = row["tot_chole"]
    return hdl_chole / total_chole

In [20]:
def get_ldl_cholestorol_ratio(row):
    ldl_chole = row["LDL_chole"]
    total_chole = row["tot_chole"]
    return ldl_chole / total_chole

In [21]:
def classify_eyesight(val):
    if val >= 1: 
        return "Good"
    elif val > 0.5: 
        return "Average"
    elif val > 0.25: 
        return "Poor"
    else:
        return "Very Poor"

In [22]:
def classify_blood_pressure(row):
    sbp = row["SBP"]
    dbp = row["DBP"]
    if sbp < 120 and dbp < 80:
        return "Normal"
    elif sbp < 130 and dbp < 80:
        return "Elevated"
    elif sbp < 140 and dbp < 90:
        return "Hypertension Stage 1"
    elif sbp < 180 and dbp < 120:
        return "Hypertension Stage 2"
    else:
        return "Hypertensive Crisis"

In [23]:
def classify_blood_glucose(row):
    blood_glucose = row["BLDS"]
    if blood_glucose < 100:
        return "Normal"
    elif blood_glucose < 125:
        return "Pre-Diabetes"
    else:
        return "Diabetes"

In [24]:
def classify_serum_creatinine(row):
    serum_creatinine = row["serum_creatinine"]
    if serum_creatinine < 2.7:
        return "Normal"
    else:
        return "Abnormal"

In [25]:
def classify_SGOT_AST(row):
    sgot_ast = row["SGOT_AST"]
    if sgot_ast < 40:
        return "Normal"
    else:
        return "Abnormal"

In [26]:
def classify_SGOT_ALT(row):
    sgot_alt = row["SGOT_ALT"]
    if sgot_alt < 40:
        return "Normal"
    else:
        return "Abnormal"

In [27]:
def classify_gamma_GTP(row):
    gamma_gtp = row["gamma_GTP"]
    sex = row["sex"]
    if sex == "Male":
        if gamma_gtp >= 11 and gamma_gtp <= 63:
            return "Normal"
        else:
            return "Abnormal"
    else:
        if gamma_gtp >= 8 and gamma_gtp <= 35:
            return "Normal"
        else:
            return "Abnormal"

In [28]:
df["bmi"] = df.progress_apply(get_bmi, axis=1)
df["BP_ratio"] = df.progress_apply(get_blood_pressure_ratio, axis=1)
df["HDL_ratio"] = df.progress_apply(get_hdl_cholestorol_ratio, axis=1)
df["LDL_ratio"] = df.progress_apply(get_ldl_cholestorol_ratio, axis=1)
df["sight_left_class"] = df["sight_left"].progress_apply(classify_eyesight)
df["sight_right_class"] = df["sight_right"].progress_apply(classify_eyesight)
df["blood_pressure_class"] = df.progress_apply(classify_blood_pressure, axis=1)
df["blood_glucose_class"] = df.progress_apply(classify_blood_glucose, axis=1)
df["serum_creatinine_class"] = df.progress_apply(classify_serum_creatinine, axis=1)
df["SGOT_AST_class"] = df.progress_apply(classify_SGOT_AST, axis=1)
df["SGOT_ALT_class"] = df.progress_apply(classify_SGOT_ALT, axis=1)
df["gamma_GTP_class"] = df.progress_apply(classify_gamma_GTP, axis=1)

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

...

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

  0%|          | 0/991320 [00:00<?, ?it/s]

In [29]:
df.to_parquet('Data/smk_drk_preprocessed.parquet')

In [35]:
mlflow.log_artifact('Data/')

Fontes:

[1] https://en.wikipedia.org/wiki/International_unit

[2] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4069781/#:~:text=Visual%20acuity%20(VA

[3] https://www.cdc.gov/bloodpressure/about.htm#:~:text=Blood%20pressure%20is%20measured%20using,your%20heart%20rests%20between%20beats.

[4] https://www.cdc.gov/bloodpressure/about.htm#:~:text=Blood%20pressure%20is%20measured%20using,your%20heart%20rests%20between%20beats.

[5] https://www.cdc.gov/cholesterol/ldl_hdl.htm#:~:text=Two%20types%20of%20lipoproteins%20carry,for%20heart%20disease%20and%20stroke.

[6] https://www.nhlbi.nih.gov/health/high-blood-triglycerides#:~:text=Triglycerides%20are%20a%20type%20of,does%20not%20need%20right%20away.

[7] https://www.mountsinai.org/health-library/tests/hemoglobin#:~:text=Hemoglobin%20is%20a%20protein%20in,is%20exchanged%20for%20carbon%20dioxide.

[8] https://www.mountsinai.org/health-library/tests/hemoglobin#:~:text=Hemoglobin%20is%20a%20protein%20in,is%20exchanged%20for%20carbon%20dioxide.

[9] https://www.mountsinai.org/health-library/tests/protein-urine-test

[10] https://www.mountsinai.org/health-library/tests/creatinine-blood-test#:~:text=Creatinine%20is%20a%20chemical%20waste,body%20entirely%20by%20the%20kidneys.

[11] https://my.clevelandclinic.org/health/diseases/17649-blood-pressure 

[12] https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2380